# Filter & extract

In [1]:
# %pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          20G  4.5G   15G  25% /
tmpfs            64M     0   64M   0% /dev
shm              64M     0   64M   0% /dev/shm
/dev/sda1        20G  4.5G   15G  25% /etc/hosts
/dev/sdb         98G   56G   38G  60% /home/jovyan/storage
tmpfs           7.9G     0  7.9G   0% /proc/acpi
tmpfs           7.9G     0  7.9G   0% /sys/firmware


## Meta data

In [3]:
import pandas as pd
from pathlib import Path

In [6]:
STORAGE = Path.home() / "storage"
METADATA = STORAGE / "meta"

TEMP = STORAGE / "temp"
PAPERS = STORAGE / "papers"

!mkdir -p {TEMP}
!mkdir -p {PAPERS} 

In [7]:
!ls {METADATA}

target_pmcid.txt


In [8]:
id_list = set(list(pd.read_csv(METADATA / 'target_pmcid.txt', names=["pmcid"])["pmcid"]))
len(id_list)

99705

## Packages

In [9]:
def get_package():
    for file in (STORAGE / "txt").rglob("*.tar.gz"):
        yield file

Unpack things 1 by 1, and filter with the pmcid

In [ ]:
ct = 0
for tar_file in get_package():
    !rm -rf {TEMP}
    !mkdir -p {TEMP}
    !tar -xzvf {tar_file} -C {TEMP} > {tar_file.stem}.log
    print(f"🚛 {tar_file}")
    !du -sh {TEMP}
    for paper in TEMP.rglob("*.txt"):
        if paper.stem in id_list:
            ct += 1
            !mv {paper} {PAPERS}/
            if ct % 200 == 0:
                print("📦" ,end="")
        else:
            paper.unlink()
    !rm {tar_file}